In [1]:
import pandas as pd
import pendulum

HOUSE_ALIAS = "oak"

df = pd.read_csv(f'data/energy_data_{HOUSE_ALIAS}.csv')
data = {
    'hour_start_s': [int(x/1000) for x in list(df['hour_start_ms'])],
    'kwh': list(df['kwh'])
}

data['hour_start_dt'] = [pendulum.from_timestamp(x, tz='America/New_York') for x in data['hour_start_s']]
data['onpeak'] = [1 if x.weekday() < 5 and x.hour in [7,8,9,10,11,16,17,18,19] else 0 for x in data['hour_start_dt']]
data['hour_start_dt'] = [x.to_datetime_string() for x in data['hour_start_dt']]

energy_df = pd.DataFrame(data)
display(energy_df)

,hour_start_s,kwh,hour_start_dt,onpeak
0,1733029200,5.34,2024-12-01 00:00:00,0
1,1733032800,0.18,2024-12-01 01:00:00,0
2,1733036400,2.17,2024-12-01 02:00:00,0
3,1733040000,7.22,2024-12-01 03:00:00,0
4,1733043600,0.08,2024-12-01 04:00:00,0
...,...,...,...,...
3257,1745056800,0.04,2025-04-19 06:00:00,0
3258,1745060400,0.04,2025-04-19 07:00:00,0
3259,1745064000,0.04,2025-04-19 08:00:00,0
3260,1745067600,0.04,2025-04-19 09:00:00,0


In [2]:
df = pd.read_csv('price_forecast_dates.csv')
data = {
    'hour_start_s': [int(x) for x in list(df['unix_s'])],
    'usd_mwh': [x+y for x,y in zip(list(df['tariff']), list(df['lmp']))],
}

data['hour_start_dt'] = [pendulum.from_timestamp(x, tz='America/New_York') for x in data['hour_start_s']]
data['onpeak'] = [1 if x.weekday() < 5 and x.hour in [7,8,9,10,11,16,17,18,19] else 0 for x in data['hour_start_dt']]
data['hour_start_cropped'] = [x for x in data['hour_start_dt'] if x >= pendulum.datetime(2025, 2, 21, tz='America/New_York')]
data['hour_start_cropped'] = [x.to_datetime_string() for x in data['hour_start_cropped']]

offset = len(data['usd_mwh']) - len(data['hour_start_cropped'])
data = {'hour_start_s': data['hour_start_s'][offset:], 'hour_start_dt': data['hour_start_cropped'], 'usd_mwh': data['usd_mwh'][offset:], 'onpeak': data['onpeak'][offset:]}

prices_df = pd.DataFrame(data)
display(prices_df)

,hour_start_s,hour_start_dt,usd_mwh,onpeak
0,1740114000,2025-02-21 00:00:00,78.78,0
1,1740117600,2025-02-21 01:00:00,84.13,0
2,1740121200,2025-02-21 02:00:00,103.74,0
3,1740124800,2025-02-21 03:00:00,78.40,0
4,1740128400,2025-02-21 04:00:00,106.42,0
...,...,...,...,...
1673,1746136800,2025-05-01 18:00:00,522.02,1
1674,1746140400,2025-05-01 19:00:00,520.20,1
1675,1746144000,2025-05-01 20:00:00,77.95,0
1676,1746147600,2025-05-01 21:00:00,67.80,0


In [3]:
merged_df = pd.merge(energy_df, prices_df, on='hour_start_dt', how='outer')

for col in merged_df.columns:
    if col.endswith('_x') and col[:-2] + '_y' in merged_df.columns:
        merged_df[col[:-2]] = merged_df[col].combine_first(merged_df[col[:-2] + '_y'])
        merged_df.drop([col, col[:-2] + '_y'], axis=1, inplace=True)

onpeak_df = merged_df[merged_df['onpeak'] == 1].dropna()
offpeak_df = merged_df[merged_df['onpeak'] == 0].dropna()
average_onpeak_price = sum(onpeak_df['usd_mwh']) / len(onpeak_df)
average_offpeak_price = sum(offpeak_df['usd_mwh']) / len(offpeak_df)

merged_df['usd_mwh'] = merged_df['usd_mwh'].fillna(
    merged_df['onpeak'].apply(lambda x: average_onpeak_price if x == 1 else average_offpeak_price)
)
merged_df.dropna(inplace=True)
display(merged_df)

merged_df['elec_cost'] = merged_df['kwh'] / 1000 * merged_df['usd_mwh'] 
onpeak_df = merged_df[merged_df['onpeak'] == 1]
offpeak_df = merged_df[merged_df['onpeak'] == 0]

average_price = sum(merged_df['usd_mwh'].dropna()) / len(merged_df.dropna())

print("-"*100)
print(f"[{HOUSE_ALIAS}] From December 1st 2024 to April 18th 2025")
print("-"*100)

# print(f"\nAverage prices:")
# print(f"- Onpeak: {round(average_onpeak_price, 2)} USD/MWh")
# print(f"- Offpeak: {round(average_offpeak_price, 2)} USD/MWh")
# print(f"- Total: {round(average_price, 2)} USD/MWh")

# print(f"\nElectricity usage:")
# print(f"- Onpeak: {round(sum(onpeak_df['kwh']), 2)} kWh")
# print(f"- Offpeak: {round(sum(offpeak_df['kwh']), 2)} kWh")
# print(f"- Total: {round(sum(merged_df['kwh'])/1000, 2)} MWh")

# print(f"\nCosts:")
# print(f"- Onpeak: {round(sum(onpeak_df['elec_cost']), 2)}")
# print(f"- Offpeak: {round(sum(offpeak_df['elec_cost']), 2)}")
# print(f"- Total: {round(sum(merged_df['elec_cost']), 2)} USD")

print(f"- Average price: {round(average_price, 2)} USD/MWh")
print(f"- Electricity used: {round(sum(merged_df['kwh'])/1000, 2)} MWh")
print(f"- Costs: {round(sum(merged_df['elec_cost']), 2)} USD")
print(f"=> Average price paid for electricity: {round(sum(merged_df['elec_cost'])/sum(merged_df['kwh'])*1000, 2)} USD/MWh")

print(f"\nSavings:")
all_energy_at_average_price = round(sum(merged_df['kwh']) * average_price /1000, 2)
all_energy_by_gridworks = round(sum(merged_df['elec_cost']), 2)
print(f"- Cost of getting the electricity delivered at average price: {all_energy_at_average_price} USD")
print(f"- Costs by GridWorks: {all_energy_by_gridworks} USD")
relative_savings = round((all_energy_at_average_price - all_energy_by_gridworks) / all_energy_at_average_price * 100, 2)
print(f"=> Savings: {all_energy_at_average_price - all_energy_by_gridworks} USD ({relative_savings}%)")

,kwh,hour_start_dt,usd_mwh,hour_start_s,onpeak
0,5.34,2024-12-01 00:00:00,70.515764,1.733029e+09,0.0
1,0.18,2024-12-01 01:00:00,70.515764,1.733033e+09,0.0
2,2.17,2024-12-01 02:00:00,70.515764,1.733036e+09,0.0
3,7.22,2024-12-01 03:00:00,70.515764,1.733040e+09,0.0
4,0.08,2024-12-01 04:00:00,70.515764,1.733044e+09,0.0
...,...,...,...,...,...
3265,0.04,2025-04-19 06:00:00,68.290000,1.745057e+09,0.0
3266,0.04,2025-04-19 07:00:00,86.130000,1.745060e+09,0.0
3267,0.04,2025-04-19 08:00:00,80.530000,1.745064e+09,0.0
3268,0.04,2025-04-19 09:00:00,78.760000,1.745068e+09,0.0


----------------------------------------------------------------------------------------------------
[oak] From December 1st 2024 to April 18th 2025
----------------------------------------------------------------------------------------------------
- Average price: 188.15 USD/MWh
- Electricity used: 10.08 MWh
- Costs: 752.28 USD
=> Average price paid for electricity: 74.63 USD/MWh

Savings:
- Cost of getting the electricity delivered at average price: 1896.57 USD
- Costs by GridWorks: 752.28 USD
=> Savings: 1144.29 USD (60.33%)


In [4]:
10080.03 * 188.15

1896557.6445000002